In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
#Set random seed so we get consistent results. 
SEED=1

In [3]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [4]:
START_EPOCH = 0

### Set the architecture to resnet 18 below

In [5]:
##########################
#Set torch architecture to resnet18
#ResNet-18 is a convolutional neural network that is 18 layers deep. 
#You can load a pretrained version of the network trained on more than a million images from the ImageNet database [1]. 
#The pretrained network can classify images into 1000 object categories, such as keyboard, mouse, pencil, and many animals. 
#As a result, the network has learned rich feature representations for a wide range of images. 
#The network has an image input size of 224-by-224. 

#See here for original paper which provides implementations research team used: https://arxiv.org/pdf/1512.03385.pdf
ARCH  = 'resnet18'
# please look up how to do that
########################
#Set our hyperparams 
#Outer training loop input: How many passes through entire training set we make. If batch size = training size, then epoch = iterations. 
EPOCHS = 50
#Optimizer input: how much to change the model in response to the estimated error each time the model weights are updated
LR = 0.1
#Optimizer input: momentum is method which helps accelerate gradients vectors in the right directions, thus leading to faster convergence to local/global minima. 
MOMENTUM = 0.9
#Optimizer input: Weight decay is a regularization technique by adding a small penalty, usually the L2 norm of the weights (all the weights of the model), to the loss function. 
#loss = loss + weight decay parameter * L2 norm of the weights. 
WEIGHT_DECAY = 0.0001
PRINT_FREQ = 256
#Batch size for training / validation loss calcs. 
TRAIN_BATCH=256
VAL_BATCH=256
#Setting the argument num_workers as a positive integer will turn on multi-process data loading with the specified number of loader worker processes.
#For CUDA: we recommend using automatic memory pinning (i.e., setting pin_memory=True), which enables fast data transfer to CUDA-enabled GPUs.
WORKERS=4

#TODO: Ensure these tie to the VM directories (volume mount for Docker container)
TRAINDIR ="/workspace/storage/train"
VALDIR = "/workspace/storage/val"

### Check if cuda is available here

In [6]:
# check if cuda is available in this cell
# if it is not available, you should not go forward!
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

### Assign your GPU below

In [7]:
# Assign your GPU in this cell(they are zero indexed)
GPU = 0

In [8]:
# set your active device to your GPU in this cell
torch.cuda.set_device(GPU)

In [9]:
# enable algorithm optimization
#benchmark mode is good whenever your input sizes for your network do not vary. This way, cudnn will look for the optimal set of algorithms for that particular configuration (hardware + input) (which takes some time). This usually leads to faster runtime.
#But if your input sizes changes at each iteration, then cudnn will benchmark every time a new size appears, possibly leading to worse runtime performances.
cudnn.benchmark = True

### Fill in the heart of the train section below

In [10]:
def train(train_loader, model, criterion, optimizer, epoch):
    #AverageMeter: Computes and stores the average and current value.
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    #Show ProgressMeter
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    ######################
    # switch model to train mode here
    model.train()
    ################

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        #####################
        # send the images to cuda device
        # send the target to cuda device
        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)


        # compute output
        output = model(images)
        # output = model ?? images
        

        # compute loss 
        # loss = criterion, output, target
        loss = criterion(output, target)


        # measure accuracy and record loss
        #topk accuracy counts a model as being accurate if the correct result is in the top n predicted class probabilities (i.e. may not be the prediction)
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        #Use AverageMeter().update to update the averages 
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        #First need to zero out gradient to prevent "exploding gradient" problem. 
        #Default is to sum or accumulate losses on backward passes (loss.backward()) bc it's convient for RNN. 
        optimizer.zero_grad()
        #Backprop: Computes gradient (dloss/dx) for every parameter x which has requires_grad = True. These are accumulated into x.grad for every parameter x. 
        #x.grad += dloss/dx   -->  x += -lr * x.grad
        loss.backward()
        #Update paremeters (weights) based on previous step
        optimizer.step()
    
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)

#### Fill in the validate section below

In [11]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            
            
            ### send the images and target to cuda
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)


            # compute output
            # output = model ??? images?
            output = model(images)

            # compute loss
            # loss  = criterion ?? output ?? target
            loss = criterion(output, target)


            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

### Save the checkpoint

In [12]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    # save the model state!
    # state ??? 
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [13]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [14]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [15]:
# if we are adjusting the LR manually use this
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    #Adjusting learning drop from 30 to 20 
    lr = LR * (0.1 ** (epoch // 20))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [16]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [17]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]
#Adjusting resnet normalizations here
resnet_mean_RGB = [0.485, 0.456, 0.406]
resnet_std_RGB = [0.229, 0.224, 0.225]

In [18]:
#Adjusting transforms with updated normalizations here (for Resnet18)
#Link: https://pytorch.org/vision/stable/models.html
normalize = transforms.Normalize(mean=resnet_mean_RGB, std=resnet_std_RGB)

In [19]:
#TODO: Determine which IMG_SIZE we need
#IMG_SIZE = 32
IMG_SIZE = 224

### Initialize the model using the architecture you selected above

In [20]:
# select the model
# model = ... 
#TODO: Determine how many classes we need 
NUM_CLASSES = 1000

model = models.__dict__[ARCH]()

model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)

In [21]:
# send the model to the cuda device.. 
model.cuda(GPU)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Instantiate the loss to cross entropy

In [22]:
# use the cross-entropy loss
criterion = nn.CrossEntropyLoss().cuda(GPU)

### Instantiate the optimizer to SGD

In [23]:
# use SGD .. use the momentum and weight decay vars
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

#### Create the learning rate scheduler

In [24]:
# https://pytorch.org/docs/stable/optim.html
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [25]:
#See here for these steps: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
transform_train = transforms.Compose([
    #Adjusting crop call and size
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(resnet_mean_RGB, resnet_std_RGB),
])

### Create the train dataset object

In [26]:
#https://stackoverflow.com/questions/49073799/pytorch-testing-with-torchvision-datasets-imagefolder-and-dataloader
train_dataset = torchvision.datasets.ImageFolder(
    root=TRAINDIR, transform=transform_train)

In [27]:
transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(resnet_mean_RGB, resnet_std_RGB),
])

In [28]:
val_dataset = torchvision.datasets.ImageFolder(
    root=VALDIR, transform=transform_val)

### Create the train dataloader

In [29]:
# fill this in
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=True,
        num_workers=WORKERS, pin_memory=True, sampler=None)

### Create the c

In [30]:
# fill this in..
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None) 

In [31]:
best_acc1 = 0

In [32]:
for epoch in range(START_EPOCH, EPOCHS):
    #See here for more: https://stackoverflow.com/questions/48324152/pytorch-how-to-change-the-learning-rate-of-an-optimizer-at-any-given-moment-no
    adjust_learning_rate(optimizer, epoch)
    
    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    #Adjusting by switching to manual scheduler (adjust_learning_rate)
    #scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))

Epoch: [0][   0/5005]	Time 10.889 (10.889)	Data  2.076 ( 2.076)	Loss 6.9955e+00 (6.9955e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
Epoch: [0][ 256/5005]	Time  0.694 ( 0.740)	Data  0.000 ( 0.008)	Loss 6.4472e+00 (6.7404e+00)	Acc@1   1.17 (  0.55)	Acc@5   3.52 (  2.17)
Epoch: [0][ 512/5005]	Time  0.766 ( 0.726)	Data  0.000 ( 0.005)	Loss 6.0694e+00 (6.4966e+00)	Acc@1   2.73 (  1.01)	Acc@5   7.42 (  3.81)
Epoch: [0][ 768/5005]	Time  0.694 ( 0.720)	Data  0.000 ( 0.004)	Loss 5.7823e+00 (6.3083e+00)	Acc@1   5.47 (  1.54)	Acc@5  14.06 (  5.51)


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Epoch: [0][1024/5005]	Time  0.698 ( 0.713)	Data  0.000 ( 0.004)	Loss 5.5849e+00 (6.1499e+00)	Acc@1   3.52 (  2.10)	Acc@5  14.06 (  7.23)
Epoch: [0][1280/5005]	Time  0.712 ( 0.711)	Data  0.000 ( 0.003)	Loss 5.4369e+00 (6.0155e+00)	Acc@1   7.03 (  2.68)	Acc@5  16.41 (  8.88)
Epoch: [0][1536/5005]	Time  0.698 ( 0.712)	Data  0.000 ( 0.003)	Loss 5.2308e+00 (5.8952e+00)	Acc@1   7.81 (  3.31)	Acc@5  20.70 ( 10.51)
Epoch: [0][1792/5005]	Time  0.753 ( 0.713)	Data  0.000 ( 0.003)	Loss 5.1421e+00 (5.7876e+00)	Acc@1   8.59 (  3.93)	Acc@5  24.61 ( 12.02)
Epoch: [0][2048/5005]	Time  0.715 ( 0.713)	Data  0.000 ( 0.003)	Loss 5.0145e+00 (5.6875e+00)	Acc@1   8.98 (  4.61)	Acc@5  22.66 ( 13.54)
Epoch: [0][2304/5005]	Time  0.706 ( 0.714)	Data  0.000 ( 0.003)	Loss 4.6706e+00 (5.5938e+00)	Acc@1  12.50 (  5.26)	Acc@5  30.86 ( 15.01)
Epoch: [0][2560/5005]	Time  0.727 ( 0.714)	Data  0.000 ( 0.003)	Loss 4.8868e+00 (5.5050e+00)	Acc@1  11.72 (  5.93)	Acc@5  24.61 ( 16.41)
Epoch: [0][2816/5005]	Time  0.710 ( 0.714

/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Epoch: [1][3072/5005]	Time  0.699 ( 0.715)	Data  0.000 ( 0.003)	Loss 3.4354e+00 (3.5948e+00)	Acc@1  30.08 ( 26.57)	Acc@5  55.08 ( 50.55)
Epoch: [1][3328/5005]	Time  0.689 ( 0.714)	Data  0.000 ( 0.003)	Loss 3.6926e+00 (3.5786e+00)	Acc@1  25.39 ( 26.82)	Acc@5  48.44 ( 50.85)
Epoch: [1][3584/5005]	Time  0.693 ( 0.712)	Data  0.000 ( 0.003)	Loss 3.4115e+00 (3.5632e+00)	Acc@1  32.42 ( 27.07)	Acc@5  54.69 ( 51.13)
Epoch: [1][3840/5005]	Time  0.750 ( 0.713)	Data  0.000 ( 0.003)	Loss 3.5594e+00 (3.5470e+00)	Acc@1  27.34 ( 27.32)	Acc@5  54.69 ( 51.44)
Epoch: [1][4096/5005]	Time  0.709 ( 0.712)	Data  0.000 ( 0.003)	Loss 3.3165e+00 (3.5314e+00)	Acc@1  32.03 ( 27.56)	Acc@5  57.81 ( 51.74)
Epoch: [1][4352/5005]	Time  0.699 ( 0.713)	Data  0.000 ( 0.002)	Loss 3.2977e+00 (3.5169e+00)	Acc@1  33.59 ( 27.80)	Acc@5  60.16 ( 52.01)
Epoch: [1][4608/5005]	Time  0.735 ( 0.712)	Data  0.000 ( 0.002)	Loss 3.2709e+00 (3.5024e+00)	Acc@1  32.03 ( 28.02)	Acc@5  51.95 ( 52.28)
Epoch: [1][4864/5005]	Time  0.705 ( 0.713

...

Epoch: [26][3584/5005]	Time  0.730 ( 0.710)	Data  0.000 ( 0.003)	Loss 1.5545e+00 (1.6993e+00)	Acc@1  66.02 ( 60.78)	Acc@5  83.20 ( 82.02)
Epoch: [26][3840/5005]	Time  0.703 ( 0.710)	Data  0.000 ( 0.003)	Loss 1.7403e+00 (1.7003e+00)	Acc@1  63.67 ( 60.76)	Acc@5  80.08 ( 82.01)
Epoch: [26][4096/5005]	Time  0.713 ( 0.710)	Data  0.000 ( 0.002)	Loss 1.6184e+00 (1.7013e+00)	Acc@1  65.62 ( 60.74)	Acc@5  82.81 ( 82.00)
Epoch: [26][4352/5005]	Time  0.709 ( 0.710)	Data  0.000 ( 0.002)	Loss 1.7780e+00 (1.7017e+00)	Acc@1  57.42 ( 60.72)	Acc@5  81.25 ( 81.99)
Epoch: [26][4608/5005]	Time  0.699 ( 0.710)	Data  0.000 ( 0.002)	Loss 1.6772e+00 (1.7022e+00)	Acc@1  58.98 ( 60.72)	Acc@5  84.77 ( 81.98)
Epoch: [26][4864/5005]	Time  0.729 ( 0.710)	Data  0.000 ( 0.002)	Loss 1.8091e+00 (1.7025e+00)	Acc@1  59.38 ( 60.71)	Acc@5  78.91 ( 81.98)
Test: [  0/196]	Time  2.181 ( 2.181)	Loss 9.3323e-01 (9.3323e-01)	Acc@1  75.39 ( 75.39)	Acc@5  94.14 ( 94.14)
 * Acc@1 64.206 Acc@5 85.776
lr: [0.1]
Epoch: [27][   0/5005]	

/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Epoch: [27][1536/5005]	Time  0.701 ( 0.712)	Data  0.000 ( 0.003)	Loss 1.5918e+00 (1.6805e+00)	Acc@1  63.28 ( 61.19)	Acc@5  82.81 ( 82.30)
Epoch: [27][1792/5005]	Time  0.686 ( 0.709)	Data  0.000 ( 0.003)	Loss 1.8490e+00 (1.6838e+00)	Acc@1  57.42 ( 61.12)	Acc@5  80.47 ( 82.24)
Epoch: [27][2048/5005]	Time  0.693 ( 0.706)	Data  0.000 ( 0.003)	Loss 1.6146e+00 (1.6833e+00)	Acc@1  64.84 ( 61.14)	Acc@5  81.25 ( 82.24)
Epoch: [27][2304/5005]	Time  0.736 ( 0.707)	Data  0.000 ( 0.003)	Loss 1.7136e+00 (1.6852e+00)	Acc@1  57.81 ( 61.08)	Acc@5  82.81 ( 82.22)
Epoch: [27][2560/5005]	Time  0.699 ( 0.707)	Data  0.000 ( 0.003)	Loss 1.7574e+00 (1.6865e+00)	Acc@1  60.16 ( 61.05)	Acc@5  82.03 ( 82.23)
Epoch: [27][2816/5005]	Time  0.700 ( 0.708)	Data  0.000 ( 0.003)	Loss 1.6491e+00 (1.6871e+00)	Acc@1  59.38 ( 61.03)	Acc@5  80.47 ( 82.21)
Epoch: [27][3072/5005]	Time  0.710 ( 0.707)	Data  0.000 ( 0.003)	Loss 1.5841e+00 (1.6876e+00)	Acc@1  61.33 ( 61.02)	Acc@5  82.81 ( 82.22)
Epoch: [27][3328/5005]	Time  0.700

Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/opt/conda/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/opt/conda/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/opt/conda/lib/python3.8/multiprocessing/connection.py", 

KeyboardInterrupt: 